In [1]:
import networkx as nx
import  numpy as np 
import bigfloat as bp


def GBPG(L):
    G=nx.Graph()
    for i in range(L**2):
        x=i//L 
        y=i%L
        G.add_node(i,attr='node',position=(x,y))
    for i in G.nodes():
        x=G.node[i]['position'][0]
        y=G.node[i]['position'][1]
        neiborposition=[((x+1)%L,y),((x-1)%L,y),(x,(y-1)%L),(x,(y+1)%L)]
        for position in neiborposition:
            x1=position[0]
            y1=position[1]
            node=x1*L+y1
            G.add_edge(i,node)
    for i,j in G.edges():
        G.add_node((i,j), attr='link')
    
    for i in G.nodes():
        if G.node[i]['attr']=='node':
            x=G.node[i]['position'][0]
            y=G.node[i]['position'][1]
            regionlist=[(x,(y+1)%L),((x+1)%L,y),((x+1)%L,(y+1)%L)]
            A,B,C=[i*L+j for i,j in regionlist]
            G.add_node((i,A,B,C),attr='region')  
    for i,j in G.edges():
        G.remove_edge(i,j)
    for i in G.nodes():
        if G.node[i]['attr']=='link':
            G.add_edge(i,i[0],attr='link to node')
            G.add_edge(i,i[1],attr='link to node')
        
    for i in G.nodes():
        if G.node[i]['attr']=='region':
            a,b,c,d=i[0],i[1],i[2],i[3]
            x1,y1=sorted((a,b))
            x2,y2=sorted((a,c))
            x3,y3=sorted((b,d))
            x4,y4=sorted((c,d))
            G.add_edge(i,(x1,y1),attr='region to link')
            G.add_edge(i,(x2,y2),attr='region to link')
            G.add_edge(i,(x3,y3),attr='region to link')
            G.add_edge(i,(x4,y4),attr='region to link')
    for i,j in G.edges():
        if G.edge[i][j]['attr']=='link to node':
            G.edge[i][j]['cavity_pro']=np.random.uniform(-0.1,0.1)*0.05
        
    for i,j in G.edges():
        if G.edge[i][j]['attr']=='region to link':
            G.edge[i][j]['cavity_pro']={i[0]:np.random.uniform(-0.1,0.1),i[1]:np.random.uniform(-0.1,0.1),i:np.random.uniform(-0.1,0.1)}    
    for i in G.nodes():
        G.node[i]['megnestim']=0
        
    return G   


In [2]:
def GBPmethod(G,T,L,J,rate):
    beta=1.0/T
    for i,j in G.edges():
        if G.edge[i][j]['attr']=='link to node':
            regiona,regionb=[m  for m in G.neighbors(j) if G.node[m]['attr']=='region']
            Node=[m for m in G.neighbors(j) if G.node[m]['attr']=='node' and m!=i ][0]
            nodebregion=[k for k in G.neighbors(Node) if G.node[k]['attr']=='link' and k!=j]
            linkA,linkB,linkC=nodebregion
            X=G.edge[regiona][j]['cavity_pro'][j]+G.edge[regionb][j]['cavity_pro'][j]+J
            Y=G.edge[Node][linkA]['cavity_pro']+G.edge[Node][linkB]['cavity_pro']+G.edge[Node][linkC]['cavity_pro']+G.edge[regiona][j]['cavity_pro'][Node]+ G.edge[regionb][j]['cavity_pro'][Node]
            Z=G.edge[regiona][j]['cavity_pro'][i]+G.edge[regionb][j]['cavity_pro'][i]
            G.edge[i][j]['cavity_pro']=rate*(Z+1/(2*beta)*bp.log(bp.cosh(beta*(X+Y))/bp.cosh(beta*(-X+Y))))+(1-rate)*G.edge[i][j]['cavity_pro']
            
           
            
            
    for a,alpha in G.edges():
        if G.edge[a][alpha]['attr']=='region to link':
            linklist=[m for m in G.neighbors(alpha) if m!=a]
            linkb,=[m for m in linklist if m[0]==a[0] or m[1]==a[0]]
            linkc,=[m for m in linklist if m[0]==a[1] or m[1]==a[1]]
            linkd,=[m for m in linklist if m!=linkc and m!=linkb]
            rho,=[m for m in G.neighbors(linkb) if G.node[m]['attr']=='region' and m!=alpha ]
            lamda,=[m for m in G.neighbors(linkc) if G.node[m]['attr']=='region' and m!=alpha ]
            eta,=[m for m in G.neighbors(linkd) if G.node[m]['attr']=='region' and m!=alpha ]
            Nodei=a[0]
            Nodej=a[1]
            Nodel,=[m for m in linkb if m!=a[0]]
            Nodek,=[m for m in linkc if m!=a[1]]
            linkd1,linkd2=[m for m in G.neighbors(Nodel) if m not in [linkb,linkd]]
            linkd3,linkd4=[m for m in G.neighbors(Nodek) if m not in [linkd,linkc]]
            X=beta*(G.edge[Nodel][linkd1]['cavity_pro']+G.edge[Nodel][linkd2]['cavity_pro']+G.edge[linkb][rho]['cavity_pro'][Nodel]+G.edge[linkd][eta]['cavity_pro'][Nodel] )   
            Y=beta*(G.edge[Nodek][linkd3]['cavity_pro']+G.edge[Nodek][linkd4]['cavity_pro']+G.edge[linkd][eta]['cavity_pro'][Nodek]+G.edge[linkc][lamda]['cavity_pro'][Nodek])
            A=beta*(J+G.edge[linkb][rho]['cavity_pro'][linkb])
            B=beta*(J+G.edge[linkc][lamda]['cavity_pro'][linkc])
            C=beta*(J+G.edge[linkd][eta]['cavity_pro'][linkd])
            KPP=2*bp.cosh(X+A+Y+B)*bp.exp(C)+2*bp.cosh(X+A-Y-B)*bp.exp(-C)
            KMM=2*bp.cosh(X-A+Y-B)*bp.exp(C)+2*bp.cosh(X-A-Y+B)*bp.exp(-C)
            KPM=2*bp.cosh(X+A+Y-B)*bp.exp(C)+2*bp.cosh(X+A-Y+B)*bp.exp(-C)
            KMP=2*bp.cosh(X-A+Y+B)*bp.exp(C)+2*bp.cosh(X-A-Y-B)*bp.exp(-C)
            G.edge[a][alpha]['cavity_pro'][Nodei]=rate*(G.edge[linkb][rho]['cavity_pro'][Nodei]-G.edge[Nodei][linkb]['cavity_pro'] +1/(4*beta)*bp.log(KPP*KPM/KMP/KMM))+(1-rate)*G.edge[a][alpha]['cavity_pro'][Nodei]
            G.edge[a][alpha]['cavity_pro'][Nodej]=rate*(G.edge[linkc][lamda]['cavity_pro'][Nodej]-G.edge[Nodej][linkc]['cavity_pro']+1/(4*beta)*bp.log(KPP*KMP/KPM/KMM))+(1-rate)*G.edge[a][alpha]['cavity_pro'][Nodej]
            G.edge[a][alpha]['cavity_pro'][a]=rate*(1/(4*beta)*bp.log(KPP*KMM/KPM/KMP))+(1-rate)*G.edge[a][alpha]['cavity_pro'][a] 
            
            
    for i in G.nodes():
        if G.node[i]['attr']=='node':
            neiborlist=G.neighbors(i)
            neibormessage=[G.edge[i][m]['cavity_pro'] for m in neiborlist]
            G.node[i]['megnestim']=bp.tanh(beta*(sum(neibormessage)))
            
    allmeg=[G.node[i]['megnestim'] for i in G.nodes() if G.node[i]['attr']=='node' ]
    G.graph['meg']=sum(allmeg)/L**2
           
            
            
    
    return G    
            

In [18]:
G=GBPG(3)

In [19]:
for i,j in G.edges():
    if G.edge[i][j]['attr']=='link to node':
        G.edge[i][j]['cavity_pro']=1
    if G.edge[i][j]['attr']=='region to link':
        G.edge[i][j]['cavity_pro'][i]=1
        G.edge[i][j]['cavity_pro'][i[0]]=1
        G.edge[i][j]['cavity_pro'][i[1]]=1
    

In [20]:
for i,j in G.edges():
    print(i,j)
    print(G.edge[i][j]['cavity_pro'])

0 (0, 3)
1
0 (0, 6)
1
0 (0, 2)
1
0 (0, 1)
1
1 (0, 1)
1
1 (1, 4)
1
1 (1, 7)
1
1 (1, 2)
1
2 (0, 2)
1
2 (1, 2)
1
2 (2, 5)
1
2 (2, 8)
1
3 (0, 3)
1
3 (3, 6)
1
3 (3, 5)
1
3 (3, 4)
1
4 (1, 4)
1
4 (3, 4)
1
4 (4, 7)
1
4 (4, 5)
1
5 (2, 5)
1
5 (3, 5)
1
5 (4, 5)
1
5 (5, 8)
1
6 (0, 6)
1
6 (3, 6)
1
6 (6, 8)
1
6 (6, 7)
1
7 (1, 7)
1
7 (4, 7)
1
7 (6, 7)
1
7 (7, 8)
1
8 (2, 8)
1
8 (5, 8)
1
8 (6, 8)
1
8 (7, 8)
1
(0, 3) (0, 1, 3, 4)
{0: 1, 3: 1, (0, 3): 1}
(0, 3) (2, 0, 5, 3)
{0: 1, 3: 1, (0, 3): 1}
(0, 6) (6, 7, 0, 1)
{0: 1, 6: 1, (0, 6): 1}
(0, 6) (8, 6, 2, 0)
{0: 1, 6: 1, (0, 6): 1}
(0, 2) (2, 0, 5, 3)
{0: 1, 2: 1, (0, 2): 1}
(0, 2) (8, 6, 2, 0)
{0: 1, 2: 1, (0, 2): 1}
(0, 1) (0, 1, 3, 4)
{0: 1, 1: 1, (0, 1): 1}
(0, 1) (6, 7, 0, 1)
{0: 1, 1: 1, (0, 1): 1}
(1, 4) (0, 1, 3, 4)
{1: 1, 4: 1, (1, 4): 1}
(1, 4) (1, 2, 4, 5)
{1: 1, 4: 1, (1, 4): 1}
(1, 7) (6, 7, 0, 1)
{1: 1, 7: 1, (1, 7): 1}
(1, 7) (7, 8, 1, 2)
{1: 1, 7: 1, (1, 7): 1}
(1, 2) (1, 2, 4, 5)
{1: 1, 2: 1, (1, 2): 1}
(1, 2) (7, 8, 1, 2)
{1: 1, 2: 1,

In [21]:
G=GBPmethod(G,2,3,1,1)

In [22]:
for i,j in G.edges():
    print(i,j)
    print(G.edge[i][j]['cavity_pro'])

0 (0, 3)
4.8734073953299237
0 (0, 6)
4.8734073953299237
0 (0, 2)
4.8734073953299237
0 (0, 1)
4.8734073953299237
1 (0, 1)
4.9999987873525633
1 (1, 4)
4.8734073953299237
1 (1, 7)
4.8734073953299237
1 (1, 2)
4.8734073953299237
2 (0, 2)
4.9999987873525633
2 (1, 2)
4.9999989315440025
2 (2, 5)
4.8734073953299237
2 (2, 8)
4.8734073953299237
3 (0, 3)
4.9999987873525633
3 (3, 6)
4.8734073953299237
3 (3, 5)
4.8734073953299237
3 (3, 4)
4.8734073953299237
4 (1, 4)
4.9999989315440025
4 (3, 4)
4.9999989315440025
4 (4, 7)
4.8734073953299237
4 (4, 5)
4.8734073953299237
5 (2, 5)
4.9999990585903120
5 (3, 5)
4.9999989315440025
5 (4, 5)
4.9999990585904470
5 (5, 8)
4.8734073953299237
6 (0, 6)
4.9999987873525633
6 (3, 6)
4.9999989315440025
6 (6, 8)
4.8734073953299237
6 (6, 7)
4.8734073953299237
7 (1, 7)
4.9999989315440025
7 (4, 7)
4.9999990585904470
7 (6, 7)
4.9999990585903120
7 (7, 8)
4.8734073953299237
8 (2, 8)
4.9999990585903120
8 (5, 8)
4.9999991705303319
8 (6, 8)
4.9999990585903120
8 (7, 8)
4.999999170

In [26]:
2+np.log(np.cosh(4)/np.cosh(1))

4.8734073953299237